# Mask on face detection

POIDVIN Hugo, LAY Frédéric, NGUYEN VAN NGUYEN Vincent, LECORFF Erwann

_____________________________________________________________________

Ce notebook contient un travail de classification de photos de profils selon 3 catégories :
- Portant un masque correctement
- Portant un masque incorrectement
- Ne portant pas de masque

En pleine pandémie de Covid-19, le port du masque apparait comme la solution la plus adéquate pour contrer la prolifération de ce virus.

Les données ont été tirées du site : https://www.kaggle.com/spandanpatnaik09/face-mask-detectormask-not-mask-incorrect-mask et télécharger en local sur nos machines.

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import glob
import matplotlib.pyplot as plt

In [8]:
absolutepath = os.path.abspath(__file__)
print(absolutepath)

fileDirectory = os.path.dirname(absolutepath)

with_mask_path = "/dataset/with_mask"
incorrect_mask_path = "/dataset/incorrect_mask"
without_mask_path = "/dataset/without_mask"
print(len(os.listdir(with_mask_path)))
print(len(os.listdir(incorrect_mask_path)))
print(len(os.listdir(without_mask_path)))

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '/dataset/with_mask'